# Aprendizaje Automático - Intermedio

## Cross validation

### Introducción

En este tutorial aprenderemos cómo usar la **validación cruzada** para medir mejor la ejecución del modelo.

El aprendizaje automático es un proceso iterativo. Te enfrentarás a elecciones sobre qué variables predictivas usar, qué tipos de modelos usar, qué argumentos proporcionar a esos modelos, etc. Hasta ahora, has tomado estas decisiones de una manera basada en datos midiendo la calidad del modelo con un conjunto de validación (o reserva).

Pero hay algunos inconvenientes en este enfoque. Para entender esto imagina que tienes un conjunto de datos con 5000 filas. Por lo general, mantendrás aproximadamente el 20% de los datos como un conjunto de datos de validación, ó 1000 filas. Pero esto deja alguna posibilidad aleatoria de determinar las puntuaciones del modelo. Es decir, un modelo podría funcionar bien en un conjunto de 1000 filas, incluso si fuera incorrecto en 1000 filas diferentes.

En el extremo, puedes imaginar tener solo 1 fila de datos en el conjunto de validación. Si comparas modelos alternativos, ¡cuál de ellos hace las mejores predicciones en un solo punto de datos será sobre todo una cuestión de suerte!

En general, cuanto mayor sea el conjunto de validación, menor aleatoriedad (también conocido como "ruido") hay en nuestra medida de calidad del modelo y más confiable será. Desafortunadamente, solo podemos obtener un conjunto de validación grande eliminando filas de nuestros datos de entrenamiento y los conjuntos de datos de entrenamiento más pequeños significan modelos peores.

### ¿Qué es la validación cruzada?

En la **validación cruzada** o *cross-validation*, ejecutamos nuestro proceso de modelado en diferentes subconjuntos de datos para obtener múltiples medidas de calidad del modelo.

Por ejemplo, podríamos comenzar dividiendo los datos en 5 partes, cada una de ella correspondiente al 20% del conjunto de datos completo. En este caso, decimos que hemos dividido los datos en 5 "**folds**".

![cross_validation](./images/cross_validation.png)

Luego, ejecutamos un experimento para cada fold:

+ En el **Experimento 1**, usamos el primer fold como un conjunto de validación (o reserva) y todo lo demás como datos de entrenamiento. Esto nos da una medida de la calidad del modelo basada en un conjunto de reserva del 20%.
+ En el **Experimento 2**, conservamos los datos del segundo fold (y usamos todo excepto el segundo fold para entrenar el modelo). El conjunto de reserva se utiliza para obtener una segunda estimación de la calidad del modelo.
+ Repetimos este proceso, usando cada fold una vez como conjunto de reserva. En conjunto, el 100% de los datos se usan como reserva en algún momento y terminamos con una medida de la calidad del modelo que se basa en todas las filas del conjunto de datos (incluso si no usamos todas las filas simultáneamente).

### ¿Cuándo debemos usar la validación cruzada?

La validación cruzada proporciona una medida más precisa de la calidad del modelo, lo cual es especialmente importante si estamos tomando muchas decisiones de modelado. Sin embargo, puede llevar más tiempo ejecutarlo, ya que estima varios modelos (uno para cada fold).

Entonces, dadas estas contrapartidas, ¿cuándo deberíamos usar cada enfoque?

+ Para *conjuntos de datos pequeños*, donde la carga computacional adicional no es un gran problema, debemos ejecutar la validación cruzada.
+ Para *conjuntos de datos más grandes*, un solo conjunto de validación es suficiente. Nuestro código se ejecutará más rápido y es posible que tengamos suficientes datos para que haya poca necesidad de reutilizar algunos de ellos para reservarlos.

No existe un umbral simple para lo que constituye un conjunto de datos grande versus pequeño. Pero si el modelo tarda unos minutos o menos en ejecutarse, probablemente valga la pena cambiar a validación cruzada.

Alternativamente, se puede ejecutar la validación cruzada y ver si las puntuaciones para cada experimento parecen cercanas. Si cada experimento produce los mismos resultados, un solo conjunto de validación es probablemente suficiente.

### Ejemplo

In [5]:
import pandas as pd

# Lee los datos
data = pd.read_csv('./input/melbourne-housing-snapshot/melb_data.csv')

# Selecciona el subconjunto de predictores
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Selecciona el objetivo
y = data.Price

Luego, definimos un pipeline que utiliza un imputador para completar los valores ausentes y un modelo de random forest para hacer predicciones.

Si bien es posible hacer una validación cruzada sin pipelines, ¡es bastante difícil! El uso de un pipeline hará que el código sea notablemente sencillo.

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])

Obtenemos las puntuaciones de validación cruzada con la función `cross_val_score()` de scikit-learn. Establecemos el número de folds con el parámetro `cv`.

In [9]:
from sklearn.model_selection import cross_val_score

# Multiplicamos por -1 ya que sklearn calcula MAE negativos
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("Puntuación MAE:\n", scores)

Puntuación MAE:
 [301628.7893587  303164.4782723  287298.331666   236061.84754543
 260383.45111427]


El parámetro `scoring` elige una medida de la calidad del modelo a usar: en este caso, elegimos el error absoluto medio negativo (MAE). Los documentos de scikit-learn muestran una [lista de opciones](https://scikit-learn.org/stable/modules/model_evaluation.html).

Es un poco sorprendente que especifiquemos MAE negativo. Scikit-learn tiene una convención en la que se definen todas las métricas para que un número alto sea mejor. El uso de negativos aquí les permite ser consistentes con esa convención, aunque el MAE negativo es casi desconocido en otros lugares.

Generalmente, queremos una sola medida de la calidad del modelo para comparar modelos alternativos. Entonces tomamos el promedio de todas los experimentos.

In [10]:
print("Puntuación media MAE (a través de los experimentos):")
print(scores.mean())

Puntuación media MAE (a través de los experimentos):
277707.3795913405


### Conclusión
El uso de la validación cruzada produce una medida mucho mejor de la calidad del modelo, con el beneficio adicional de hacer nuestro código más limpio: ten en cuenta que ya no necesitamos seguir la pista de conjuntos de entrenamiento y validación separados. Entonces, especialmente para conjuntos de datos pequeños, ¡es una buena mejora!